## Libraries and variables

In [1]:
import h2o
from h2o.automl import H2OAutoML
import pandas as pd

data_path = "../01_Data/"
file_name_class = '4-1_psp-data_feateng_all_class.csv'
# file_name_class = '4-1_psp-data_feateng_featsel_class.csv'
# file_name_class = '4-1_psp-data_feateng_pcb_class.csv'

file_path_class = data_path + file_name_class

## Data load

In [2]:
df = pd.read_csv(file_path_class, index_col=0)
df

,success,amount_sc,3D_secured,transaction_hour_sc,card_Diners,card_Master,card_Visa,PSP_Goldcard,PSP_Moneycard,PSP_Simplecard,PSP_UK_Card
0,0,0.133013,0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
1,1,0.133013,0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
2,0,0.371795,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
3,1,0.371795,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0,0.189103,0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
50324,0,0.655449,0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
50325,0,0.136218,0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
50326,0,0.136218,0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
50327,0,0.136218,0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0


## AutoML - Find baseline model

In [3]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
; OpenJDK 64-Bit Server VM (build 22+36-2370, mixed mode, sharing)
  Starting server from C:\Program Files\Python312\Lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\boehm\AppData\Local\Temp\tmpmlvcjaok
  JVM stdout: C:\Users\boehm\AppData\Local\Temp\tmpmlvcjaok\h2o_boehm_started_from_python.out
  JVM stderr: C:\Users\boehm\AppData\Local\Temp\tmpmlvcjaok\h2o_boehm_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,02 secs
H2O_cluster_timezone:,Europe/Berlin
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.2
H2O_cluster_version_age:,1 month and 20 days
H2O_cluster_name:,H2O_from_python_boehm_7big52
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.945 Gb
H2O_cluster_total_cores:,0
H2O_cluster_allowed_cores:,0
H2O_cluster_status:,"locked, healthy"


In [5]:
psp_df = h2o.import_file(file_path_class)

psp_train, psp_test, psp_valid = psp_df.split_frame(ratios=[.7, .15])

y = "success"
x = psp_df.columns
x.remove(y)
x.remove('C1')

psp_train[y] = psp_train[y].asfactor()
psp_test[y] = psp_test[y].asfactor()

aml = H2OAutoML(max_models=20, seed =10, exclude_algos=['StackedEnsemble', 'DeepLearning'], verbosity='info', nfolds=0)

aml.train(x=x, y=y, training_frame=psp_train, validation_frame=psp_valid)


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
AutoML progress: |
21:31:30.155: Project: AutoML_2_20240703_213130
21:31:30.155: Cross-validation disabled by user: no fold column nor nfolds > 1.
21:31:30.156: Setting stopping tolerance adaptively based on the training frame: 0.005334566353389347
21:31:30.156: Build control seed: 10
21:31:30.156: training frame: Frame key: AutoML_2_20240703_213130_training_py_13_sid_af67    cols: 12    rows: 35140  chunks: 32    size: 522416  checksum: -7428413560442138496
21:31:30.156: validation frame: Frame key: py_12_sid_af67    cols: 12    rows: 7688  chunks: 32    size: 134604  checksum: 864545729229181820
21:31:30.156: leaderboard frame: Frame key: py_12_sid_af67    cols: 12    rows: 7688  chunks: 32    size: 134604  checksum: 864545729229181820
21:31:30.156: blending frame: NULL
21:31:30.156: response column: success
21:31:30.156: fold column: null
21:31:30.156: weights column: null
21:31:30.156: Au

Model Details
=============
H2OGradientBoostingEstimator : Gradient Boosting Machine
Model Key: GBM_1_AutoML_2_20240703_213130


Model Summary: 
    number_of_trees    number_of_internal_trees    model_size_in_bytes    min_depth    max_depth    mean_depth    min_leaves    max_leaves    mean_leaves
--  -----------------  --------------------------  ---------------------  -----------  -----------  ------------  ------------  ------------  -------------
    35                 35                          86394                  13           15           14.8857       153           212           191.657

ModelMetricsBinomial: gbm
** Reported on train data. **

MSE: 0.1430297202319776
RMSE: 0.37819270251021186
LogLoss: 0.44826901902826205
Mean Per-Class Error: 0.33031549729298926
AUC: 0.7383417079526404
AUCPR: 0.42715873880847144
Gini: 0.47668341590528085

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.2443259852831691
       0      1      Error    Rate
-----  -----  -----  -------  -----------------
0      20738  7252   0.2591   (7252.0/27990.0)
1      2871   4279   0.4015   (2871.0/7150.0)
Total  23609  11531  0.2881   (10123.0/35140.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       0.244326     0.458113  207
max f2                       0.132728     0.610354  310
max f0point5                 0.298679     0.442439  160
max accuracy                 0.417588     0.804411  86
max precision                0.74136      1         0
max recall                   0.0501959    1         396
max specificity              0.74136      1         0
max absolute_mcc             0.272665     0.294389  182
max min_per_class_accuracy   0.221135     0.671383  228
max mean_per_class_accuracy  0.211627     0.672418  237
max tns                      0.74136      27990     0
max fns                      0.74136      7148      0
max fps                      0.0440549    27990     399
max tps                      0.0501959    7150      396
max tnr                      0.74136      1         0
max fnr                      0.74136      0.99972   0
max fpr                      0.0440549    1         399
max tpr                      0.0501959    1         396

Gains/Lift Table: Avg response rate: 20,35 %, avg score: 20,40 %
group    cumulative_data_fraction    lower_threshold    lift      cumulative_lift    response_rate    score      cumulative_response_rate    cumulative_score    capture_rate    cumulative_capture_rate    gain      cumulative_gain    kolmogorov_smirnov
-------  --------------------------  -----------------  --------  -----------------  ---------------  ---------  --------------------------  ------------------  --------------  -------------------------  --------  -----------------  --------------------
1        0.0100171                   0.549873           3.71394   3.71394            0.755682         0.609074   0.755682                    0.609074            0.0372028       0.0372028                  271.394   271.394            0.0341303
2        0.0200057                   0.496325           2.81439   3.26481            0.57265          0.521096   0.664296                    0.565148            0.0281119       0.0653147                  181.439   226.481            0.0568831
3        0.0300228                   0.457284           2.51319   3.01403            0.511364         0.477505   0.61327                     0.535906            0.0251748       0.0904895                  151.319   201.403            0.0759129
4        0.0400114                   0.425548           2.67437   2.92924            0.54416          0.440721   0.596017                    0.512143            0.0267133       0.117203                   167.437   192.924            0.0969098
5        0.05                        0.402903           2.31032   2.80559            0.470085         0.413484   0.570859   

In [6]:
lb = aml.leaderboard

In [7]:
print(lb)

model_id                                           auc    logloss     aucpr    mean_per_class_error      rmse       mse
GBM_1_AutoML_2_20240703_213130                0.656697   0.477515  0.329098                0.375817  0.390507  0.152495
GBM_5_AutoML_2_20240703_213130                0.656335   0.47568   0.329856                0.372946  0.389699  0.151866
GBM_grid_1_AutoML_2_20240703_213130_model_7   0.656019   0.476875  0.328267                0.375204  0.390307  0.15234
GBM_2_AutoML_2_20240703_213130                0.655256   0.476898  0.328136                0.374661  0.390355  0.152377
GBM_grid_1_AutoML_2_20240703_213130_model_5   0.654884   0.476751  0.328812                0.374146  0.390255  0.152299
GBM_grid_1_AutoML_2_20240703_213130_model_12  0.654565   0.477584  0.329102                0.377587  0.39042   0.152428
GBM_grid_1_AutoML_2_20240703_213130_model_6   0.654365   0.477742  0.329729                0.377523  0.390579  0.152552
GBM_grid_1_AutoML_2_20240703_213130_model

In [8]:
psp_pred = aml.leader.predict(psp_test)
psp_pred.head()

gbm prediction progress: |███████████████████████████████████████████████████████| (done) 100%


predict,p0,p1
0,0.874884,0.125116
0,0.921554,0.0784465
1,0.783429,0.216571
0,0.937543,0.0624572
1,0.729216,0.270784
1,0.729216,0.270784
1,0.791311,0.208689
0,0.926473,0.0735271
0,0.855316,0.144684
1,0.742635,0.257365


In [10]:
aml.leader.model_performance(psp_test)

ModelMetricsBinomial: gbm
** Reported on test data. **

MSE: 0.1514585214352034
RMSE: 0.3891767226276559
LogLoss: 0.4750151956416389
Mean Per-Class Error: 0.3696317662606865
AUC: 0.6661385633674606
AUCPR: 0.3537750547321943
Gini: 0.33227712673492116

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.19489428999971586
       0     1     Error    Rate
-----  ----  ----  -------  ---------------
0      3406  2569  0.43     (2569.0/5975.0)
1      472   1054  0.3093   (472.0/1526.0)
Total  3878  3623  0.4054   (3041.0/7501.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       0.194894     0.4094    247
max f2                       0.122035     0.568862  320
max f0point5                 0.320865     0.363067  131
max accuracy                 0.455748     0.802426  64
max precision                0.724233     1         0
max recall                   0.0504549    1         397
max specificity              0.724233     1         0
max absolute_mcc             0.194894     0.210044  247
max min_per_class_accuracy   0.21251      0.624509  229
max mean_per_class_accuracy  0.194894     0.630368  247
max tns                      0.724233     5975      0
max fns                      0.724233     1525      0
max fps                      0.0446798    5975      399
max tps                      0.0504549    1526      397
max tnr                      0.724233     1         0
max fnr                      0.724233     0.999345  0
max fpr                      0.0446798    1         399
max tpr                      0.0504549    1         397

Gains/Lift Table: Avg response rate: 20,34 %, avg score: 20,47 %
group    cumulative_data_fraction    lower_threshold    lift      cumulative_lift    response_rate    score      cumulative_response_rate    cumulative_score    capture_rate    cumulative_capture_rate    gain      cumulative_gain    kolmogorov_smirnov
-------  --------------------------  -----------------  --------  -----------------  ---------------  ---------  --------------------------  ------------------  --------------  -------------------------  --------  -----------------  --------------------
1        0.010132                    0.550607           3.36321   3.36321            0.684211         0.60498    0.684211                    0.60498             0.034076        0.034076                   236.321   236.321            0.0300593
2        0.0201306                   0.503026           2.62158   2.99485            0.533333         0.525555   0.609272                    0.565531            0.0262123       0.0602883                  162.158   199.485            0.0504139
3        0.0301293                   0.466236           2.55604   2.84923            0.52             0.484085   0.579646                    0.538502            0.025557        0.0858453                  155.604   184.923            0.0699458
4        0.040128                    0.431368           2.42496   2.74352            0.493333         0.448299   0.55814                     0.516026            0.0242464       0.110092                   142.496   174.352            0.0878323
5        0.0501266                   0.40723            1.63849   2.5231             0.333333         0.417879   0.513298                    0.496449            0.0163827       0.126474                   63.8488   152.31             0.0958468
6        0.10012                     0.341032           1.61227   2.06829            0.328            0.367776   0.420772                    0.432198            0.0806029       0.207077                   61.2273   106.829            0.134274
7        0.150113                    0.308902           1.4943    1.87713            0.304            0.323212   0.381883                    0.395902            0.0747051       0.281782                   49.4301   87.7131            0.165297
8        0.200107      

In [11]:
h2o.shutdown(prompt=False)

H2O session _sid_af67 closed.


C:\Users\boehm\AppData\Local\Temp\ipykernel_24508\1954269801.py:1: H2ODeprecationWarning: Deprecated, use ``h2o.cluster().shutdown()``.
  h2o.shutdown(prompt=False)
